In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("metrics.csv")
evals = df[df["role"] == "eval"]
plt.plot(evals["round"], evals["eval_loss"], label="Eval loss")
plt.plot(evals["round"], evals["perplexity"], label="Perplexity")
plt.legend()
plt.show()
